In [51]:
import numpy as np
import scipy as sp

In [52]:
from quadrotor.dynamics import QuadrotorDynamicsBase, QuadrotorState
from quadrotor.controller import ControllerBase, QuadrotorCommands
from quadrotor.trajectory import TrajectoryBase, TrajectoryState

In [53]:
from sym import Rot3

def dRot3(R: Rot3, omega: np.ndarray) -> Rot3:
    """
    NOTE(roy): If interested, we can discuss the math here later. There are other
    ways of doing this but to keep the ode_int formulation, we specifically require
    dq/dt. Refer to 'Baseile Graf - Quaternions and Dynamics' or (for intuition)
    https://math.stackexchange.com/questions/1896379/how-to-use-the-quaternion-derivative
    """
    quat = np.array(R.to_storage())  # Convert to raw quaternion
    (q0, q1, q2, q3) = quat  # xyzw storage, see https://symforce.org/api-gen-cpp/class/classsym_1_1Rot3.html

    G = np.array([[ q3,  q2, -q1, -q0],
                [-q2,  q3,  q0, -q1],
                [ q1, -q0,  q3, -q2]])
    quat_dot = (G.T @ omega) / 2
    return Rot3.from_storage(quat_dot)

In [54]:
from dataclasses import dataclass, field

@dataclass
class Params:
    """A little utility class to hold our quadrotor parameters"""
    mass: float = 1.352  # [kg]

    # NOTE(roy) this is a matrix now as discussed!
    # NOTE(roy) don't get scared by the field/default_factory thing! It's just a way to do default field initialization for dataclasses,
    # see https://docs.python.org/3/library/dataclasses.html. Don't worry about it!
    inertia: np.ndarray = field(default_factory=lambda: np.array([
        [9.8e-3, 0, 0],
        [0, 10.02e-3, 0],
        [0, 0, 18.6e-3],
    ]))

    inertia_inverse: np.ndarray = field(default_factory=lambda: np.linalg.inv(
        np.array([
        [9.8e-3, 0, 0],
        [0, 10.02e-3, 0],
        [0, 0, 18.6e-3],
    ])))


    rotor_diameter: float = 10 * 0.0254  # [m] 10 inches

    static_thrust_coefficient: float = 0.14553  # [-] Thrust coefficient
    static_torque_coefficient: float = 0.01047  # [-] Torque coefficient

    # NOTE(roy) assume this is the same front-to-back and left-to-right
    arm_length: float = 0.3814 / 2.0  # [m] Arm length

    # Universal constants
    g: float = 9.80665  # [m/s2] Gravity
    rho: float = 1.225  # [kg/m3] Sea-level density

    def rotor_model(self, static_coefficient: float) -> float:
        # NOTE(roy) I'm just including this for completeness sake, this means we can e.g. see how air density
        # influences our simulation. Refer to this source for derivation:
        # https://web.mit.edu/16.unified/www/FALL/thermodynamics/notes/node86.html
        return self.rho * static_coefficient * (self.rotor_diameter ** 4) / (4 * np.pi ** 2)

    # NOTE(roy) if you're unsure what @property decorators do: they make it so that we can call k_thrust as if it was just a class
    # variable. I.e. we'd write params.k_thrust now, rather than params.k_thrust()
    @property
    def k_thrust(self):
        return self.rotor_model(self.static_thrust_coefficient)

    @property
    def k_torque(self):
        return self.rotor_model(self.static_torque_coefficient)

# Simply treat as a global
p = Params()
mixing_matrix = np.array([
            [1, 1, 1, 1],
            [0, p.arm_length, 0, -p.arm_length],
            [-p.arm_length, 0, p.arm_length, 0],
            [p.k_torque / p.k_thrust, -p.k_torque / p.k_thrust, p.k_torque / p.k_thrust, -p.k_torque / p.k_thrust]
        ])

### The dynamics

In [55]:
# Copy your dynamics here!

class FullQuadrotorDynamics(QuadrotorDynamicsBase):
    def __init__(self) -> None:
        super().__init__()

    @staticmethod
    def rotor_thrust_model(rotor_rates: np.ndarray) -> np.ndarray:
        # NOTE(roy): Implement the simple rotor trust model we discussed in the slides
        # (For fun) to see how complex this can get, see for example: https://arxiv.org/pdf/1601.00733.pdf
        return p.k_thrust * rotor_rates ** 2

   #Implement the mixing matrix (i.e. compute u1 and u2 from the rotor rates)
    
    
    def step(self, t: float, input: QuadrotorCommands) -> QuadrotorState:
        F1, F2, F3, F4 = FullQuadrotorDynamics.rotor_thrust_model(input.rotor_rates) 

        # NOTE(roy) Implement the thrust/torque calculation we discussed in the slides
        thrust_force_vector = np.array([F1, F2, F3, F4])
        u_matrix = mixing_matrix @ thrust_force_vector
        u1 = u_matrix[0]
        u2 = u_matrix[1:] 

        # NOTE(roy) this local function is a wrapper around the class state_derivative function
        # all this does is convert the our state dataclass from/into a vector that the scipy
        # integration solver can work with. See dynamics.py for the definition of these
        # from/to state vector functions.
        def state_derivative_local_wrapped(t: float, state: np.ndarray) -> np.ndarray:
            return FullQuadrotorDynamics.state_derivative(
                QuadrotorState.from_state_vector(state),
                u1,
                u2,
            ).to_state_vector()

        state_vector = self.state.to_state_vector()
        #raise NotImplementedError("Use the scipy function solve_ivp to integrate to the next timestep")
        

        # NOTE(roy) solution = sp.integrate.solve_ivp(....
        solution = sp.integrate.solve_ivp(
            fun=state_derivative_local_wrapped,
            t_span=[0, self.dt],
            y0=state_vector,
            method='RK45'
        )

        self.state = QuadrotorState.from_state_vector(solution['y'][:, -1])
        return self.state

    #Modify the state_derivative functions to take a vector for u2 rather than a float (because now of course that's 3 numbers!)
    #Modify the accel formulation in your model to 3D (see equations in slides)
    #Modify the angular_accel formulation in your model to 3D (see equations in slides)
    @staticmethod
    def state_derivative(state: QuadrotorState, u1: float, u2: np.ndarray) -> QuadrotorState:
        rotation_body_to_world = state.orientation.to_rotation_matrix()
        gravity_vector = np.array([0, 0, -p.mass * p.g])
        u1_vector = np.array([0, 0, u1])
        accel = (gravity_vector + np.matmul(rotation_body_to_world, u1_vector)) / p.mass
        angular_accel = np.matmul(p.inertia_inverse, 
                                  (u2 - np.cross(state.angular_velocity, 
                                                                    np.matmul(p.inertia, 
                                                                              state.angular_velocity))))
        
        # NOTE(roy) this might look a little confusing at first glance, but we're populating DERIVATIVES
        # of the state here! So position -> velocity, velocity -> acceleration, etc...
        # If you can think of a more readable way to do this (e.g. defining a QuadrotorStateDerivative class)
        # feel free to open a pull-request into the upstream repository!
        state_derivative = QuadrotorState(
            position=state.velocity,
            velocity=accel,
            orientation=dRot3(state.orientation, state.angular_velocity),
            angular_velocity=angular_accel,
        )

        return state_derivative


### The controller

In [56]:
@dataclass
class ControllerParams:
    """The same as above but for the controller gains"""

    # NOTE(roy) diag just means we get a matrix with these values on the diagonal,
    # so these are 3x3 matrices (print them if you're unsure!)
    # We use this formulation because it makes the math a bit more readable later on!
    K_p: np.ndarray = field(default_factory=lambda: np.diag([1, 1, 100]))
    K_d: np.ndarray = field(default_factory=lambda: np.diag([0.5, 0.5, 10]))

    # Added gains for the attitude controller
    Ka_p: np.ndarray = field(default_factory=lambda: np.diag([300, 300, 50]))
    Ka_d: np.ndarray = field(default_factory=lambda: np.diag([50, 50, 20]))

    # Min/max rotor rates!
    rotor_rate_min: float = 180
    rotor_rate_max: float = 600

controller_p = ControllerParams()

In [57]:
# Write your controller here!


class Controller(ControllerBase):
    @staticmethod
    def rotor_rates_from_u(u: np.ndarray) -> np.ndarray:
        # NOTE(roy) implement your conversion from u (so that's [u1, u2_1, u2_2, u2_3] like we discussed, a 4-sized vector)
        # to rotor rates (which is also a four-sized vector)

        # NOTE(roy) the steps here are:
        #  1. convert u -> forces with the inverse mixing matrix
        #  2. convert forces -> rotor rates by inverting the rotor rate model
        #  3. limit the rotor rates with rotor_rate_min/rotor_rate_max above!
        #raise NotImplementedError("rotor_rates_from_u not implemented!")

        forces = np.linalg.inv(mixing_matrix) @ u
        # print('forces = ', forces)
        forces = np.maximum(forces, 0) # replaces negative forces with 0 for squareroot below
        rotor_rates = np.sqrt(forces / p.k_thrust)
        for i in np.arange(len(rotor_rates)): # forces below min and above max changed to min and max rotor rates
            if rotor_rates[i] > controller_p.rotor_rate_max:
                rotor_rates[i] = controller_p.rotor_rate_max
            if rotor_rates[i] < controller_p.rotor_rate_min:
                rotor_rates[i] = controller_p.rotor_rate_min
        # print('rotor_rates = ', rotor_rates)
        return rotor_rates

    def step(self, t: float, trajectory: TrajectoryState, state: QuadrotorState) -> QuadrotorCommands:
        desired_position, desired_velocity = trajectory.position, trajectory.velocity
        actual_position, actual_velocity = state.position, state.velocity
        
        # NOTE(roy) first compute the desired acceleration using the equation from the lectures!
        # I already extracted the values you need to compute it above
        state_error = desired_position - actual_position
        state_error_derivative = desired_velocity - actual_velocity
        commanded_acceleration = controller_p.K_d @ state_error_derivative + controller_p.K_p @ state_error  # Your equation here!

        # NOTE(roy) now compute u_1 from the commanded acceleration using the linearized equation from the lectures!
        u_1 = p.mass * (p.g + commanded_acceleration[2])  # Your equation here!


    # Expand the controller class
    # - Compute the desired roll/pitch using the equations from the lecture
    # - Compute the attitude errors
    # - Compute the desired angular acceleration
    # - Compute `u_2`
    # - Tune your system (the gains provided _should_ provide a relatively stable starting point!)

        yaw = state.orientation.to_yaw_pitch_roll()[0]
        desired_roll_pitch_matrix = np.array([[np.cos(yaw), np.sin(yaw)],
                                            [np.sin(yaw), -np.cos(yaw)]])
        desired_pitch, desired_roll = np.transpose(desired_roll_pitch_matrix) @ commanded_acceleration[0:2]
        #print("desired pitch = ", desired_pitch)    
        #print("desired roll = ", desired_roll)

        angle_orientation_error = np.array([desired_roll, desired_pitch, trajectory.yaw]) - np.flip(state.orientation.to_yaw_pitch_roll())
        for i in np.arange(len(angle_orientation_error)):
            angle_orientation_error[i] = (angle_orientation_error[i] + np.pi) % (2 * np.pi) - np.pi #keep in domain of -pi to pi
        # print("angle orientation error = ", angle_orientation_error)
        # print("state ang vel =", state.angular_velocity)
        angular_velocity_error = np.array([0, 0, trajectory.yaw_rate])   - state.angular_velocity
        # print("angular velocity error = ", angular_velocity_error)
        u_2 = p.inertia @ (controller_p.Ka_d @ angular_velocity_error + controller_p.Ka_p @ angle_orientation_error)
        #u_2 = np.zeros(3)
        u = np.array([u_1, *u_2])

        rotor_rates = Controller.rotor_rates_from_u(u)
        return QuadrotorCommands(rotor_rates)

### The trajectory

In this notebook, you'll take your working controller and start generating some fun trajectories for it! I've provided a small `Waypoint` class which describes a single point for the quadrotor to visit. Then I've also provided a `WaypointTrajectory` class that takes in a list of those waypoints. I already wrote some of the code such that for each time `t` we know which waypoint comes before and after it.

It's your job to generate smooth (and fast!) trajectories between the waypoints. What I would do to start:
1. Generate the simplest possible trajectory and just **interpolate** the position between the previous and next waypoint.
2. Generate a bang-coast-bang trajectory with a given maximum velocity- and acceleration
3. (If you get here you're already doing great!)
4. Generate a smooth polynomial trajectory! This is tricky and I don't expect you to get to this at all - you will also need to precompute the trajectory if you follow the method from the slides. Happy to help if you do have questions about this!

In [58]:
@dataclass
class Waypoint:
    time: float  # Time to arrive at waypoint
    
    position: np.ndarray  # xyz-position
    yaw: float = 0.0

    speed: np.ndarray = field(default_factory=lambda: np.zeros(3))  # xyz-velocity

In [59]:
def bang_coast_bang_trajectory(prev_waypoint, next_waypoint, t):
    # Extract relevant variables
    t0, t1 = prev_waypoint.time, next_waypoint.time
    pos0, pos1 = prev_waypoint.position, next_waypoint.position
    max_speed = 1.0  # Adjust this value as needed
    max_acceleration = 0.5  # Adjust this value as needed

    # Total distance to cover
    total_distance = np.linalg.norm(pos1 - pos0)

    # Determine the time needed to reach max_speed under max_acceleration
    t_accel = max_speed / max_acceleration

    # Distance covered during acceleration and deceleration phases
    dist_accel = 0.5 * max_acceleration * t_accel**2

    # Check if there is enough distance for a full bang-coast-bang profile
    if 2 * dist_accel > total_distance:
        # Not enough distance: use a triangular (bang-bang) trajectory instead
        t_accel = np.sqrt(total_distance / max_acceleration)
        t_coast = 0
    else:
        # Enough distance: calculate the time spent in the coast phase
        dist_coast = total_distance - 2 * dist_accel
        t_coast = dist_coast / max_speed

    t1_accel_end = t0 + t_accel
    t2_coast_end = t1_accel_end + t_coast

    if t <= t1_accel_end:
        # Acceleration phase
        tau = t - t0
        acceleration = max_acceleration
        position = pos0 + 0.5 * acceleration * tau**2 * ((pos1 - pos0) / total_distance)
        velocity = acceleration * tau * ((pos1 - pos0) / total_distance)
    elif t <= t2_coast_end:
        # Coast phase
        tau = t - t1_accel_end
        position = pos0 + dist_accel * ((pos1 - pos0) / total_distance) + max_speed * tau * ((pos1 - pos0) / total_distance)
        velocity = max_speed * ((pos1 - pos0) / total_distance)
    else:
        # Deceleration phase
        tau = t - t2_coast_end
        acceleration = -max_acceleration
        position = pos1 - 0.5 * acceleration * tau**2 * ((pos1 - pos0) / total_distance)
        velocity = max_speed + acceleration * tau * ((pos1 - pos0) / total_distance)

    return position, velocity

In [60]:
import typing as T

class WaypointTrajectory(TrajectoryBase):
    def __init__(self, waypoints: T.List[Waypoint]) -> None:
        self.waypoints = waypoints
        assert self.waypoints[0].time == 0, "First waypoint has to have time 0!"

        # Doublecheck the waypoints are in order:
        for a, b in zip(self.waypoints[:-1], self.waypoints[1:]):
            assert b.time > a.time, "Waypoints time needs to be increasing!"
    
    def eval(self, t: float):
        # If we're at the last waypoint just pick the last way point
        next_waypoint = next((w for w in self.waypoints if w.time >= t), None)
        if next_waypoint is None:
            return TrajectoryState(t, position=self.waypoints[-1].position, yaw=self.waypoints[-1].yaw)

        # Else find the waypoints we're in between right now!
        prev_waypoint = self.waypoints[self.waypoints.index(next_waypoint) - 1]

        # And now that you have prev_waypoint and next_waypoint - generate a trajectory for a time somewhere in between!
        #raise NotImplementedError()
        # Generate the trajectory for the time somewhere in between
        position, velocity = bang_coast_bang_trajectory(prev_waypoint, next_waypoint, t)

        # Linearly interpolate the yaw angle
        yaw = np.interp(t, [prev_waypoint.time, next_waypoint.time], [prev_waypoint.yaw, next_waypoint.yaw])

        return TrajectoryState(t, position=position, velocity=velocity, yaw=yaw)

In [61]:
# NOTE(roy) this just writes an R now. Make it write something fun!
trajectory = WaypointTrajectory([
    Waypoint(0.0, position=np.zeros(3)),
    Waypoint(2.0, position=np.array([0, 0, 1])),
    Waypoint(4.0, position=np.array([0, 0.5, 1])),
    Waypoint(6.0, position=np.array([0, 0.5, 0.5])),
    Waypoint(8.0, position=np.array([0, 0.0, 0.5])),
    Waypoint(10.0, position=np.array([0, 0.5, 0.0])),
])

### Running the simulator

In [62]:
from quadrotor.simulator import SimulatorBase, SimulatorState

# This might be slow to run the first time!
from quadrotor.renderer import animate_k3d

In [63]:
def run_and_render(initial_state: QuadrotorState, t_total: int = 2.0):
    sim = SimulatorBase(
        dt=0.005,
        dynamics=FullQuadrotorDynamics(),
        controller=Controller(),
        trajectory=trajectory,
        initial_state=initial_state,
        t_total=t_total,
    )
    
    # Run the simulator
    output = sim.simulate()

    # K3D renderer
    plot = animate_k3d(output)
    
    plot.display()
    plot.start_auto_play()

In [64]:
# NOTE(roy) play with this variable and see what happens!
initial_state = QuadrotorState(
    position=np.zeros(3),
    velocity=np.array([0, 0, 0]),
    orientation=Rot3.from_yaw_pitch_roll(0, 0, 0),
    angular_velocity=np.zeros(3),
)

run_and_render(initial_state, 10.0)

Output()

In [65]:

import typing as T

class WaypointTrajectoryMinJerk(TrajectoryBase):
    def __init__(self, waypoints: T.List[Waypoint]) -> None:
        self.waypoints = waypoints
        assert self.waypoints[0].time == 0, "First waypoint has to have time 0!"

        # Doublecheck the waypoints are in order:
        for a, b in zip(self.waypoints[:-1], self.waypoints[1:]):
            assert b.time > a.time, "Waypoints time needs to be increasing!"
    
    def eval(self, t: float) -> T.Tuple[float, float, float]:
        # Find the segment in which time t lies
        for i in range(len(self.waypoints) - 1):
            if self.waypoints[i].time <= t <= self.waypoints[i + 1].time:
                wp_start = self.waypoints[i]
                wp_end = self.waypoints[i + 1]
                break
        else:
            # If time t is beyond the last waypoint, return the last waypoint position
            wp_end = wp_start = self.waypoints[-1]
            return (wp_end.position, wp_end.velocity, 0.0)

        T = wp_end.time - wp_start.time
        tau = (t - wp_start.time) / T

        # Construct the boundary conditions vector
        boundary_conditions = np.array([
            wp_start.position,            # Initial position
            wp_end.position,              # Final position
            wp_start.velocity * T,        # Initial velocity scaled by T
            wp_end.velocity * T,          # Final velocity scaled by T
            wp_start.acceleration * T**2, # Initial acceleration scaled by T^2
            wp_end.acceleration * T**2    # Final acceleration scaled by T^2
        ])

        # Construct the minimum jerk matrix
        M = np.array([
            [1, 0, 0, 0, 0, 0],
            [1, 1, 1, 1, 1, 1],
            [0, 1, 0, 0, 0, 0],
            [0, 1, 2, 3, 4, 5],
            [0, 0, 2, 0, 0, 0],
            [0, 0, 2, 6, 12, 20]
        ])

        # Solve for the polynomial coefficients
        coefficients = np.linalg.solve(M, boundary_conditions)

        # Construct the time powers vector for position, velocity, and acceleration
        tau_vector = np.array([1, tau, tau**2, tau**3, tau**4, tau**5])
        tau_vector_deriv = np.array([0, 1, 2 * tau, 3 * tau**2, 4 * tau**3, 5 * tau**4])
        tau_vector_double_deriv = np.array([0, 0, 2, 6 * tau, 12 * tau**2, 20 * tau**3])

        # Calculate position, velocity, and acceleration
        position = np.dot(coefficients, tau_vector)
        velocity = np.dot(coefficients, tau_vector_deriv) / T
        acceleration = np.dot(coefficients, tau_vector_double_deriv) / T**2

        return TrajectoryState(t, position=position, velocity=velocity)

In [66]:
trajectory = WaypointTrajectoryMinJerk([
    Waypoint(0.0, position=np.zeros(3)),
    Waypoint(2.0, position=np.array([0, 0, 1])),
    Waypoint(4.0, position=np.array([0, 0.5, 1])),
    Waypoint(6.0, position=np.array([0, 0.5, 0.5])),
    Waypoint(8.0, position=np.array([0, 0.0, 0.5])),
    Waypoint(10.0, position=np.array([0, 0.5, 0.0])),
])

In [67]:
def run_and_render(initial_state: QuadrotorState, t_total: int = 2.0):
    sim = SimulatorBase(
        dt=0.005,
        dynamics=FullQuadrotorDynamics(),
        controller=Controller(),
        trajectory=trajectory,
        initial_state=initial_state,
        t_total=t_total,
    )
    
    # Run the simulator
    output = sim.simulate()

    # K3D renderer
    plot = animate_k3d(output)
    
    plot.display()
    plot.start_auto_play()

In [68]:
# NOTE(roy) play with this variable and see what happens!
initial_state = QuadrotorState(
    position=np.zeros(3),
    velocity=np.array([0, 0, 0]),
    orientation=Rot3.from_yaw_pitch_roll(0, 0, 0),
    angular_velocity=np.zeros(3),
)

run_and_render(initial_state, 10.0)

AttributeError: 'Waypoint' object has no attribute 'velocity'

### Looking at the controller behavior
If all is well, the quadrotor should follow the trajectory!

Try some experiments:
- Can you make the controller more/less damped and faster/slower to respond?
- Can you write a trajectory that has feasible velocities/positions?
    - Hint: the trajectory that's currently there is infeasible, why?

### Commit your updated notebook to your fork!